In [1]:
import poppy
import jax
import jax.numpy as jnp
import functools
import galois
import numpy as np

In [2]:
p = 17
n = 3
q = p ** n
f = poppy.field(p,n)
d = 100
a = poppy.random((d,d), f, poppy.SEED)
b = poppy.random((d,d), f, poppy.SEED)
c = poppy.random(1, f, poppy.SEED)
a, c

(array shape (1, 100, 100).
 field order  4913.,
 array shape (1, 1, 1).
 field order  4913.)

In [3]:
ai = a.inv()
ar = a.reciprocal()
bi = b.inv()
ai, bi, ar
%timeit a.inv().lift.block_until_ready()

103 ms ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
print(a.lift[0, :10])
(a-a@ai@a@bi@b).lift

[ 9  7 14  7 11  7  0  5  9 12]


Array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [6]:
%timeit a.reciprocal().lift.block_until_ready( )
%timeit a.trace().lift.block_until_ready( )
%timeit a.det().block_until_ready()
%timeit a.lu()[0].block_until_ready()

377 ms ± 36.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
841 µs ± 65.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
42.1 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
34.6 ms ± 927 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
GF = galois.Field( q )
ag = GF( np.array( a.proj( ) ) )
#%timeit np.reciprocal( ag )
%timeit np.trace( ag )
#np.reciprocal( ag )

102 µs ± 6.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
%timeit a.trace( ).lift.block_until_ready( )

647 µs ± 50.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [18]:
an = np.array( a.lift )
%timeit np.trace( an )

9.06 µs ± 89 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [19]:
ag = GF( np.array( a.proj( ) ) )
%timeit np.trace( ag )

170 µs ± 5.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
N  = 10000
d  = 2
p  = 7
p1 = 13
p2 = 17
n  = 6
q  = p ** n
f  = poppy.field( p,  n )
F  = poppy.field( p1, d )
FF = poppy.field( p2, n )
a  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 0 ), ( N, F.n, F.n ),            0,       q, dtype = poppy.DTYPE )
AA = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 1 ), ( F.n * FF.n, F.n * FF.n ), 0, 2 ** 32, dtype = poppy.DTYPE )
BB = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 2 ), ( F.n * FF.n, F.n * FF.n ), 0, 2 ** 32, dtype = poppy.DTYPE )
A  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 1 ), ( FF.n, FF.n ),             0, 2 ** 32, dtype = poppy.DTYPE )
B  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 2 ), ( FF.n, FF.n ),             0, 2 ** 32, dtype = poppy.DTYPE )

@jax.jit
def poppyhash( a ):
    b = poppy.proj( AA @ a + BB, F )
    c = poppy.proj( A @ b + B, FF )
    return c % ( 4 * N )

liftbatch = jax.vmap( poppy.lift, in_axes = ( 0, None ) )
hashbatch = jax.vmap( poppyhash )

ap = liftbatch( a, f )
h  = hashbatch( ap ).squeeze( )
au = jnp.unique( a, axis = 0 )
hu = jnp.unique( h )
len( au ), len( hu ), F.q, FF.q

(10000, 8832, 169, 24137569)